In [2]:
from netCDF4 import Dataset
import numpy as np

In [2]:
df = Dataset('/scratch/local1/hzg2/schout_300.nc', 'r')

In [5]:
df.variables['spm_very_fine_silt']

<class 'netCDF4._netCDF4.Variable'>
float32 spm_very_fine_silt(nSCHISM_hgrid_node)
    units: kg/l
    long_name: Suspended sediment concentration of very_fine_silt
    description: Suspended sediment concentration based on BAWs UnTrim model using the Dietrich model received from Arne Hammrich. Average of 3D data over time and depth
unlimited dimensions: 
current shape = (32432,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [13]:
kg_per_liter = 0.00030795527166685096
kg_per_m3 = kg_per_liter * 1000

radius_spm = 6e-6
radius_particle = 5e-4 #0.0005
volume_particle = 4/3 * np.pi * radius_spm **3
density_particle = 2650
mass_particle = density_particle * volume_particle
particles_per_kg = 1 / mass_particle

cross_section = np.pi * (radius_particle + radius_spm)**2

avg_vertical_velocity = 0.0798

In [14]:
particles_per_kg

417072702022.78656

In [8]:
particles_per_m3*cross_section*avg_vertical_velocity

0.014246131353355947

In [4]:
cross_section

8.043608166545165e-07

In [5]:
## terminal velocity
alpha = 2/9
rho_particle = 2650
rho_water = 1000
g = 9.81
radius_particle = 1e-3
viscosity = 1e-3



terminal_velocity = alpha * (rho_particle - rho_water)/ viscosity * g * radius_particle**2 

# print equaton with filles values
print(f'{terminal_velocity} = {alpha} * ({rho_particle} - {rho_water})/ {viscosity} * {g} * {radius_particle}**2')

3.597 = 0.2222222222222222 * (2650 - 1000)/ 0.001 * 9.81 * 0.001**2


## Acceleration

In [7]:
import numpy as np

# Given values
rho_p = 2650  # density of the particle in kg/m^3 (particle)
rho_f = 1000  # density of the fluid in kg/m^3 (water)
g = 9.81  # acceleration due to gravity in m/s^2
r = 1e-3  # radius of the particle in meters (1mm)
C_d = 0.47  # assumed drag coefficient for a sphere

# Calculate the volume and mass of the particle
V = (4/3) * np.pi * R**3  # volume of the sphere
m_p = rho_p * V  # mass of the particle


# Calculate the acceleration for each velocity
def buoyancy_induced_accelerations(velocities):
    return g * ((rho_p - rho_f) / rho_p) - (3 * rho_f * velocities**2 * C_d) / (8 * r * rho_p)




In [19]:
buoyancy_induced_accelerations(0.01)

6.1014622641509435


$ g \left( \frac{{\rho_p - \rho_f}}{{\rho_p}} \right) - \frac{{3 \rho_f \cdot \text{{velocities}}^2 \cdot C_d}}{{8 R \cdot \rho_p}}$


In [8]:
v_init = 0
t_max = 60 #seconds
t_step = 1 # second

# calculate the particle velocity at each time step
t = np.arange(0, t_max, t_step)
v = np.zeros_like(t)
v[0] = v_init

for i in range(1, len(t)):
    a = buoyancy_induced_accelerations(v[i-1])
    v[i] = v[i-1] + a * t_step

# calculate the particle position at each time step
x = np.zeros_like(t)
x[0] = 0
for i in range(1, len(t)):
    x[i] = x[i-1] + v[i-1] * t_step


/tmp/ipykernel_10563/3190169335.py:17: RuntimeWarning: overflow encountered in scalar multiply
  return g * ((rho_p - rho_f) / rho_p) - (3 * rho_f * velocities**2 * C_d) / (8 * R * rho_p)
/tmp/ipykernel_10563/2955086182.py:18: RuntimeWarning: overflow encountered in scalar add
  x[i] = x[i-1] + v[i-1] * t_step


-1.3354531909978477e+17